In [1]:
import folium
import folium.plugins

from matplotlib import pyplot as plt
import pandas as pd
import xml.etree.ElementTree as ET

import random

net_xml = "seattle.net.xml"
add_xml = "seattle.add.xml"
rou_xml = "seattle.trips.xml"

In [2]:
parking_areas = []

root_add = ET.parse(add_xml).getroot()
for child in root_add.iter('parkingArea'):
    parking_areas.append(child.get('id'))
len(parking_areas)

44

In [3]:
edges_all = []

root_net = ET.parse(net_xml).getroot()
for child in root_net.iter('edge'):
    if child.get('function') != 'internal':
        edges_all.append(child.get('id'))
len(edges_all)

55

In [4]:
edges_od = []
edges_od.extend([i for i in edges_all if i[3:5] == '00'])
edges_od.extend([i for i in edges_all if i[3:5] == '04'])
edges_od.extend([i for i in edges_all if i[3:5] == '23'])
edges_od.extend([i for i in edges_all if i[3:5] == '27'])
len(edges_od)

21

In [5]:
edges_o = ['01-23-NW', '25-00-NE', '26-00-NE', '27-00-NE', '01-27-SE', '02-27-SE', 
           '03-27-SE', '27-04-SW', '04-23-NW', '03-23-NW']
edges_d = [i for i in edges_od if i not in edges_o]

In [18]:
edges_o

['01-23-NW',
 '25-00-NE',
 '26-00-NE',
 '27-00-NE',
 '01-27-SE',
 '02-27-SE',
 '03-27-SE',
 '27-04-SW',
 '04-23-NW',
 '03-23-NW']

In [17]:
edges_d

['25-00-SW',
 '27-00-SW',
 '24-04-NE',
 '26-04-NE',
 '27-04-NE',
 '01-23-SE',
 '02-23-SE',
 '03-23-SE',
 '01-27-NW',
 '03-27-NW',
 '04-27-NW']

In [6]:
# edge_od_pairs = []
# from itertools import permutations 
# for i in list(permutations(edges_od, 2)): 
#     edge_od_pairs.append(i)

In [49]:
with open(rou_xml, "w") as rou:
    print("""<?xml version="1.0" encoding="UTF-8"?>
          <routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">
          """, 
          file=rou)

In [50]:
with open(rou_xml, "a") as rou:
    print('\t<vType id="passenger" vClass="passenger"></vType>', 
          file=rou)

In [51]:
with open(rou_xml, "a") as rou:
    # via="e1 e23 e7"
    for i in range(len(edges_o)):
        o = edges_o[i]
        d = [item for item in edges_d if item[:2] == o[:2] and item[:5] != o[:5]]
        if len(d) >= 1:
            d = d[0]
        else:
            d = [item for item in edges_d if item[:2] == str(int(o[:2])+1)][0]
        print('\t<flow id="f{}" begin="{}" end="{}" number="{}" from="{}" to="{}"/>'.format(i+200, 0, 2400, 
                                                                                           300, 
                                                                                           o, d), 
              file=rou)

In [52]:
with open(rou_xml, "a") as rou:
    route_num = 3600
    for i in range(route_num):
        pa = random.choice(parking_areas) # parking area
        o = random.choice(edges_o)
        d = random.choice(edges_d)
#         print('<trip id={}, depart="{}", from="{}", to={}>')
        print('''\t<trip id="t{}" depart="{}" from="{}" to="{}">
		 <stop index="0" parkingArea="{}" duration="{}" parking="true"/>
	</trip>'''.format(i, i*10, o, d, pa, 600), file = rou)
#         print('\t<trip id="{}" type="passenger" edges="{} {}">\n\t\t<stop index="0" parkingArea="{}" duration="20" parking="true"/>\n\t</route>'.format(i, o, d, pa), file=rou)

In [53]:
with open(rou_xml, "a") as rou:
    print('\n</routes>', file = rou)